### 1단계 : 유동인구 특성 클러스터링 

##### 1. 전처리 

In [ ]:
# 동코드 추출
gs_dong_code = pd.read_excel(data_path +'offer/*****.xlsx',
                             sheet_name='참고)구_행정동코드', skiprows=1)
seoul_code_list = gs_dong_code[gs_dong_code.시명=='서울특별시'][['행정동명', '행정동코드']]

In [ ]:
# 동별 면적 추출
area = pd.read_excel(data_path +'untact service related features.xlsx', usecols=['행정동', '면적(m^2)'])

In [ ]:
# 동코드-면적
code_area = pd.merge(seoul_code_list, area, left_on='행정동명', right_on='행정동')[['행정동코드','면적(m^2)']]

In [ ]:
code_area.head()

In [ ]:
# 동별 유동인구 데이터
pop_data = pd.DataFrame()

for i in range(19,21):
    for j in range(2,6):
        temp = pd.read_csv(data_path+'data.seoul.go.kr/LOCAL_PEOPLE_DONG_20'+str(i)+'0'+str(j)+'.csv', index_col=False)
        temp.set_index('기준일ID', inplace=True)
        temp.index = pd.to_datetime(temp.index, format='%Y%m%d')
        
        pop_data = pd.concat([pop_data, temp])

In [ ]:
# 데이터 정리
# 노원구, 중구 데이터만 추출
pop_data = pop_data[pop_data.행정동코드.isin(seoul_code_list.행정동코드)]

# 연령대별로 정리, 성별 유동인구 합치기
col_name = ['0', '10', '15', '20', '25', '30', '35', '40', '45', '50', '55', '60', '65', '70+']
m_peo = pop_data.iloc[:,3:17]
f_peo = pop_data.iloc[:,17:]
m_peo.columns = col_name
f_peo.columns = col_name
a_peo = m_peo + f_peo

pop_data_tmp = pd.concat([pop_data.iloc[:,:2], a_peo], axis=1).reset_index()

###  70세 이상 인구 제거

In [ ]:
pop_result = pop_data_tmp.sort_values(by = ['행정동코드', '기준일ID', '시간대구분'])
pop_result = pop_result.drop(['70+'], axis=1)
pop_result.reset_index(drop=True, inplace=True)

In [ ]:
pop_result.head()

###  '행정동-일별'로 평면화

In [ ]:
# 행정동-특정 날짜에 대해 유동인구 데이터는 시간대별, 연령대별로 2차원 형태
# 시간대, 연령대 - (24, 13) -> (312,) 로 변환

In [ ]:
idx_cols = ['행정동코드', '기준일ID']
num_cols = [i for i in range(0,312)]
cols = idx_cols+num_cols

In [ ]:
from tqdm import tqdm

In [ ]:
reshape_list = []

for i in tqdm(range(0,len(pop_result),24)):
    code_date = pop_result.iloc[i,[2,0]]
    pop_values = pop_result.iloc[i:i+24,3:].values.reshape(-1)
    temp_row = np.concatenate([code_date, pop_values])
    reshape_list.append(temp_row)

final = pd.DataFrame(reshape_list, columns=['행정동코드', '기준일ID']+[i for i in range(312)])

In [ ]:
final.head()

In [ ]:
final.info()

In [ ]:
final.to_csv(data_path+'checkpoints.csv', index=False, encoding='utf-8-sig')

In [ ]:
# checkpoint
final = pd.read_csv(data_path+'checkpoints.csv')
final['기준일ID'] = pd.to_datetime(final.기준일ID, format='%Y-%m-%d')
final.head()

#### 각 행정동에 대해 19/20년도 요일 평균 유동인구 계산

- 형태
      소공동 월요일 : 0~10세0001 / 0~10세0102 ... / 65~70세2324
      소공동 화요일 : 0~10세0001 / 0~10세0102 ... / 65~70세2324


In [ ]:
X_19 = final[final.기준일ID < '2020/02/01']
X_20 = final[final.기준일ID >= '2020/02/01']

In [ ]:
X_19_week = pd.DataFrame(columns=np.concatenate([['code', 'weekday'], X_19.columns[2:].values]))
X_20_week = pd.DataFrame(columns=np.concatenate([['code', 'weekday'], X_20.columns[2:].values]))

In [ ]:
len(final.행정동코드.unique()) * 7

In [ ]:
for code in final.행정동코드.unique():
    # 19년도
    temp = X_19[X_19.행정동코드 == code].drop('행정동코드', axis=1)
    temp['weekday'] = temp.기준일ID.dt.dayofweek
    week_temp = temp.groupby('weekday').mean().reset_index()
    week_temp['code'] = code
    X_19_week = X_19_week.append(week_temp)
    
    # 20년도
    temp = X_20[X_20.행정동코드 == code].drop('행정동코드', axis=1)
    temp['weekday'] = temp.기준일ID.dt.dayofweek
    week_temp = temp.groupby('weekday').mean().reset_index()
    week_temp['code'] = code
    X_20_week = X_20_week.append(week_temp)
    
X_19_week.reset_index(drop=True, inplace=True)
X_20_week.reset_index(drop=True, inplace=True)

In [ ]:
X_19_week.head()

In [ ]:
X_20_weekzhead()

In [ ]:
# 유동인구 데이터를 0과 1사이로 정규화
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def getScaled_pop(week):
    result_scale = pd.DataFrame(columns=week.columns)
    
    for i in range(len(week)):
        temp_values = week.iloc[i,2:].values.reshape(-1,1)
        scaler = MinMaxScaler()
        temp_scale = scaler.fit_transform(temp_values)
        
        row_scale = np.concatenate([week.iloc[i,:2].values, temp_scale.reshape(-1)])
        result_scale.loc[i, :]= row_scale
    
    return result_scale

In [ ]:
X_19_week_scale = getScaled_pop(X_19_week)
X_20_week_scale = getScaled_pop(X_20_week)

### 행정동 면적을 사용하여 정규화

In [ ]:
# 유동인구 데이터를 0과 1사이로 정규화
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 기존 유동인구 데이터에 면적 column 추가
def getScaled_area(week, code_area):
    temp = pd.merge(week, code_area, left_on='code', right_on='행정동코드')

    # 유동인구 / 해당 동 면적
    for i in range(2,314):
        temp[temp.columns[i]]= temp.iloc[:,i]/temp.loc[:,'면적(m^2)']
    temp.drop(['행정동코드', '면적(m^2)'], axis=1, inplace=True)

    temp_values = temp.iloc[:,2:]
    scaler = MinMaxScaler()
    temp_scale = scaler.fit_transform(temp_values)

    result_scale = pd.concat([temp.iloc[:,:2],pd.DataFrame(temp_scale, columns=temp.columns[2:])], axis=1)
    
    return result_scale

In [ ]:
X_19_week_scale = getScaled_area(X_19_week, code_area)
X_20_week_scale = getScaled_area(X_20_week, code_area)

In [ ]:
X_19_week_scale

In [ ]:
X_20_week_scale

## 2. 요일별 행정동 유동인구 시각화

In [ ]:
# '1-# 각 행정동에 대해 19/20년도 요일 평균 유동인구 계산' 참고
# X_19_week, X_20_week

In [ ]:
# weekday
import calendar

col_name = ['0', '10', '15', '20', '25', '30', '35', '40', '45', '50', '55', '60', '65']

In [ ]:
def getWeekPlot(dongname, year, colorbar):
    if year == 19 : 
        data = X_19_week
    else :
        data = X_20_week
        
    vmin = int(round(data.iloc[:,2:].min().min()))
    vmax = int(round(data.iloc[:,2:].max().max()))
    
    code = seoul_code_list[seoul_code_list.행정동명 == dongname].행정동코드.values[0]
    temp = data[data.code == code]
    
    
    n = 7
    fig = plt.figure(figsize=(20,15))
    for i in range(0,n):
        
        # 24x13 행렬도 쪼갬 
        
        pick = pd.DataFrame(temp.iloc[i,2:].to_numpy().reshape(24,13))
        pick.index.name = '시간대구분'
        pick.columns = col_name
        output= pick.T.astype(float) # type error : not numeric?
        
        ax = fig.add_subplot(3,3,i+1)
        
        if colorbar:
            plt.pcolor(output, cmap='YlGnBu', vmin=vmin, vmax=vmax)  # colorbar 통일 
        else :
            plt.pcolor(output, cmap='YlGnBu')
        plt.xticks(np.arange(0.5, len(output.columns), 1), output.columns)
        plt.yticks(np.arange(0.5, len(output.index), 1), output.index)
        plt.title(dongname+'_'+str(year)+'_'+calendar.day_name[i])
        plt.colorbar()

In [ ]:
getWeekPlot('소공동', 19, True)

## 3. 차원축소(TSNE) 및 클러스터링

###  연도별 TSNE, DBSCAN, KMEANS 수행

In [ ]:
x_19_train = X_19_week_scale.iloc[:,2:].to_numpy()
x_20_train = X_20_week_scale.iloc[:,2:].to_numpy()

In [ ]:
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

In [ ]:
# learning_rate를 일반적 범위인 10~1000까지 살펴가면 최적값 찾음
for i in range(10,1000, 1):
    model = TSNE(learning_rate=i, random_state=0, n_jobs=-1)
    transformed = model.fit_transform(x_20_train)

    plt.scatter(transformed[:,0], transformed[:,1], cmap='prism')
    plt.show()

In [ ]:
def pltTSNE_DBSCAN(x, lr, eps, min_s, rand):
    model = TSNE(learning_rate=lr, random_state=rand)
    transformed = model.fit_transform(x)

    model = DBSCAN(eps=eps, min_samples=min_s) #metric='euclidean'
    predict = model.fit(transformed)
    y_pred = predict.labels_
        
    # Assign result to df
    dataset = pd.DataFrame({'Column1':transformed[:,0],'Column2':transformed[:,1]})
    dataset['cluster_num'] = pd.Series(y_pred)

    plt.scatter(dataset.Column1, dataset.Column2, c=dataset['cluster_num'], cmap=plt.cm.RdYlGn)
    plt.show()
    
    return dataset

In [ ]:
def pltTSNE_KMEANS(x, lr, k, rand, rand_k):
    model = TSNE(learning_rate=lr, random_state=rand)
    transformed = model.fit_transform(x)
    
    model = KMeans(init="k-means++", n_clusters=k, random_state=rand_k)
    model.fit(transformed)
    y_pred = model.labels_
    
    # Assign result to df
    dataset = pd.DataFrame({'Column1':transformed[:,0],'Column2':transformed[:,1]})
    dataset['cluster_num'] = pd.Series(y_pred)

    plt.scatter(dataset.Column1, dataset.Column2, c=dataset['cluster_num'], cmap=plt.cm.RdYlGn)
    plt.show()
    
    return dataset

In [ ]:
# 20년 : DBSCAN의 최적 eps와 min_s는 각각 3, 10임.
# 19년 : DBSCAN의 최적 eps와 min_s는 각각 3, 10임.
## eps 범위 : 0 ~ 무한대
## min_s 범위 : 2 ~ (case-1)

temp_set = pltTSNE_DBSCAN(x_20_train, 30, 3, 10, 0)
print(temp_set.cluster_num.value_counts())
#pltTSNE_DBSCAN(x_19_train, 30, 3, 10, 0)

In [ ]:
temp_set = pltTSNE_KMEANS(x_20_train, 30, 5, 0, 0)
print(temp_set.cluster_num.value_counts())
#pltTSNE_KMEANS(x_19_train, 200, 7, 20, 24)

In [ ]:
model_tNSE_2020_final = TSNE(learning_rate=30, random_state=0)
transformed_tSNE = model_tNSE_2020_final.fit_transform(x_20_train)

model_DBSCAN_2020_final = DBSCAN(eps=3, min_samples=10) #metric='euclidean'
predict_2020_final = model_DBSCAN_2020_final.fit(transformed_tSNE)
cluster_labels_2020_final = predict_2020_final.labels_

for i in range(len(cluster_labels_2020_final)):
    cluster = cluster_labels_2020_final[i]
    dong = seoul_code_list[['행정동명','행정동코드']].set_index('행정동코드').loc[X_20_week_scale.iloc[i]['code']][0]
    weekday = X_20_week_scale.iloc[i]['weekday']
    
    print(dong, '_', str(weekday), '  ', cluster)

In [ ]:
model_tNSE_2020_final = TSNE(learning_rate=30, random_state=0)
transformed_tSNE = model_tNSE_2020_final.fit_transform(x_20_train)

model_KMEANS_2020_final = KMeans(init="k-means++", n_clusters=5, random_state=0)
predict_2020_final = model_KMEANS_2020_final.fit(transformed_tSNE)
cluster_labels_2020_final = predict_2020_final.labels_

for i in range(len(cluster_labels_2020_final)):
    cluster = cluster_labels_2020_final[i]
    dong = seoul_code_list[['행정동명','행정동코드']].set_index('행정동코드').loc[X_20_week_scale.iloc[i]['code']][0]
    weekday = X_20_week_scale.iloc[i]['weekday']
    
    print(dong, '_', str(weekday), '  ', cluster)

###  실루엣 스코어 평가

In [ ]:
def visualize_silhouette_DBSCAN(X_features, lr, eps, min_s, rand): 
    
    from sklearn.datasets import make_blobs
    from sklearn.cluster import MeanShift
    from sklearn.cluster import estimate_bandwidth
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math
    
    model = TSNE(learning_rate=lr, random_state=rand)
    transformed = model.fit_transform(X_features)

    model = DBSCAN(eps=eps, min_samples=min_s) #metric='euclidean'
    predict = model.fit(transformed)
    cluster_labels = predict.labels_

    plt.plot(figsize=(4, 4), nrows=1)
    
    #print(cluster_labels)
    
    no_clu = np.where(cluster_labels != -1)
    
    X_features = X_features[no_clu]
    cluster_labels = cluster_labels[no_clu]
    #print(cluster_labels)

    sil_avg = silhouette_score(X_features, cluster_labels)
    sil_values = silhouette_samples(X_features, cluster_labels)
    n_cluster = len(np.unique(cluster_labels))
    sil_std = float(pd.DataFrame(data=[sil_values,cluster_labels]).transpose().rename(columns={0:'sil', 1:'cluster'})\
            .groupby('cluster').mean().std())

    y_lower = 10
    plt.title('Number of Cluster : '+ str(n_cluster)+'\n' \
              'Ave. Silhouette Score :' + str(round(sil_avg,3))+'\n' \
              'Std. Silhouette Score :' + str(round(sil_std,3)))
    
    plt.xlabel("The silhouette coefficient values")
    plt.ylabel("Cluster label")
    plt.xlim([-0.1, 1])
    plt.ylim([0, len(X_features) + (n_cluster + 1) * 10])
    plt.yticks([])  # Clear the yaxis labels / ticks
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1])

    # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
    for i in range(n_cluster):
        ith_cluster_sil_values = sil_values[cluster_labels==i]
        ith_cluster_sil_values.sort()

        size_cluster_i = ith_cluster_sil_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_cluster)
        plt.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                            facecolor=color, edgecolor=color, alpha=0.7)
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10

    plt.axvline(x=sil_avg, color="red", linestyle="--")

In [ ]:
model = TSNE(learning_rate=30, random_state=0)
transformed = model.fit_transform(x_20_train)

model = DBSCAN(eps=3, min_samples=10) #metric='euclidean'
predict = model.fit(transformed)
cluster_labels = predict.labels_

len(np.unique(cluster_labels))

In [ ]:
print('first : 2020 noReduction')
print('second : 2020 reduction')
visualize_silhouette_DBSCAN(x_20_train, 30, 3, 10, 0)

In [ ]:
### 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 실루엣 계수를 면적으로 시각화한 함수 작성
def visualize_silhouette_KMEANS(cluster_lists, X_features, lr, rand, rand_k): 
    
    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math
    
    model = TSNE(learning_rate=lr, random_state=rand)
    transformed = model.fit_transform(X_features)    
    
    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산.
        
        model = KMeans(init="k-means++", n_clusters=n_cluster, random_state=rand_k)
        model.fit(transformed)
        cluster_labels = model.labels_
    
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        sil_std = float(pd.DataFrame(data=[sil_values,cluster_labels]).transpose().rename(columns={0:'sil', 1:'cluster'})\
                        .groupby('cluster').mean().std())
        
        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Ave. Silhouette Score :' + str(round(sil_avg,3))+'\n' \
                          'Std. Silhouette Score :' + str(round(sil_std,3)))
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])  # * 10
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()
            
            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10
            
        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")

###  output

In [ ]:
X_19_week_01scale = X_19_week.iloc[:,2:]
max_val = X_19_week.iloc[:,2:].max(axis=1)

for i in range(len(X_19_week_01scale)):
    X_19_week_01scale.iloc[i] = X_19_week_01scale.iloc[i] / max_val[0]

X_19_week_01scale['code'] = X_19_week['code']
X_19_week_01scale['weekday'] = X_19_week['weekday']


X_20_week_01scale = X_20_week.iloc[:,2:]
max_val = X_20_week.iloc[:,2:].max(axis=1)

for i in range(len(X_20_week_01scale)):
    X_20_week_01scale.iloc[i] = X_20_week_01scale.iloc[i] / max_val[0]

X_20_week_01scale['code'] = X_20_week['code']
X_20_week_01scale['weekday'] = X_20_week['weekday']

In [ ]:
x_19_train_01scale = X_19_week_01scale.iloc[:,:-2].to_numpy()
x_20_train_01scale = X_20_week_01scale.iloc[:,:-2].to_numpy()

In [ ]:
# lr 10 ~ 1000
lr_list = [390, 400, 410]
#range(99,601,50)

for i in lr_list:
    print(i)
    model = TSNE(learning_rate=i, random_state=0, n_jobs=-1)
    transformed = model.fit_transform(x_20_train_01scale)

    plt.scatter(transformed[:,0], transformed[:,1], cmap='prism')
    plt.show()

In [ ]:
data = x_20_train_01scale

eps = 3.3
min_s = 10
lr = 400

pltTSNE_DBSCAN(data, lr, eps, min_s, 0)



visualize_silhouette_DBSCAN(data, lr, eps, min_s, 0)



model_tNSE_2020_final = TSNE(learning_rate=lr, random_state=0)
transformed_tSNE = model_tNSE_2020_final.fit_transform(data)

model_DBSCAN_2020_final = DBSCAN(eps=eps, min_samples=min_s) #metric='euclidean'
predict_2020_final = model_DBSCAN_2020_final.fit(transformed_tSNE)
cluster_labels_2020_final = predict_2020_final.labels_

for i in range(len(cluster_labels_2020_final)):
    cluster = cluster_labels_2020_final[i]
    dong = seoul_code_list[['행정동명','행정동코드']].set_index('행정동코드').loc[X_20_week_01scale.iloc[i]['code']][0]
    weekday = X_20_week_01scale.iloc[i]['weekday']
    
    print(dong, '_', str(weekday), '  ', cluster)

In [ ]:
data = x_19_train_01scale

eps = 2
min_s = 10
lr = 400

pltTSNE_DBSCAN(data, lr, eps, min_s, 0)



visualize_silhouette_DBSCAN(data, lr, eps, min_s, 0)



model_tNSE_2020_final = TSNE(learning_rate=lr, random_state=0)
transformed_tSNE = model_tNSE_2020_final.fit_transform(data)

model_DBSCAN_2020_final = DBSCAN(eps=eps, min_samples=min_s) #metric='euclidean'
predict_2020_final = model_DBSCAN_2020_final.fit(transformed_tSNE)
cluster_labels_2020_final = predict_2020_final.labels_

for i in range(len(cluster_labels_2020_final)):
    cluster = cluster_labels_2020_final[i]
    dong = seoul_code_list[['행정동명','행정동코드']].set_index('행정동코드').loc[X_20_week_01scale.iloc[i]['code']][0]
    weekday = X_20_week_01scale.iloc[i]['weekday']
    
    print(dong, '_', str(weekday), '  ', cluster)

In [ ]:
model_tNSE_2020_final = TSNE(learning_rate=lr, random_state=0)
transformed_tSNE = model_tNSE_2020_final.fit_transform(x_20_train_01scale)

model_DBSCAN_2020_final = DBSCAN(eps=eps, min_samples=min_s) #metric='euclidean'
predict_2020_final = model_DBSCAN_2020_final.fit(transformed_tSNE)
cluster_labels_2020_final = predict_2020_final.labels_

for i in range(len(cluster_labels_2020_final)):
    cluster = cluster_labels_2020_final[i]
    dong = seoul_code_list[['행정동명','행정동코드']].set_index('행정동코드').loc[X_20_week_01scale.iloc[i]['code']][0]
    weekday = X_20_week_01scale.iloc[i]['weekday']
    
    print(dong, '_', str(weekday), '  ', cluster)